# Analisis de Sentimientos

Es una rama de la clasificación de texto automatizadas que busca inferir la actitud de un usuario respecto de un tema en función de la connotación positiva o negativa. En este v

# Importo librerias

In [115]:
import numpy as np
import pandas as pd
import glob #Para leer path de los archivos

from sklearn.model_selection import  train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix as cm
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings("ignore")

# Carga de datos

In [116]:
#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

files= glob.glob(r'/kaggle/input/datasets-tweets-/*.xls*')

df= pd.DataFrame()
for i in range(0,len(files)):
            df_1 = pd.read_excel(files[i])
            df = pd.concat([df_1,df])
df

,Message,Sentimiento
0,YA ANDA CURAO VIDAL XD,Neutro
1,Ctm vidal tiene 6 dedos,Neutro
2,Me carga vidal,Neutro
3,Qué falsa resulta esta publicidad. Qué tiene ...,Neutro
4,Cuando estas un poquito hinchado por la falopa :v,Neutro
...,...,...
1490,Muy buena la App veo todo el fútbol en vivo,Positivo
1491,Muy bueno,Positivo
1492,@cryssoft @DIRECTVGO Yo la tengo y funciona bien.,Positivo
1493,Me encanta mi @DIRECTVGO 😍 donde voy tengo tod...,Positivo


In [117]:
df_test= df[df.Sentimiento.isna()]
df_train= df[~df.Sentimiento.isna()]
df_train

,Message,Sentimiento
0,YA ANDA CURAO VIDAL XD,Neutro
1,Ctm vidal tiene 6 dedos,Neutro
2,Me carga vidal,Neutro
3,Qué falsa resulta esta publicidad. Qué tiene ...,Neutro
4,Cuando estas un poquito hinchado por la falopa :v,Neutro
...,...,...
1490,Muy buena la App veo todo el fútbol en vivo,Positivo
1491,Muy bueno,Positivo
1492,@cryssoft @DIRECTVGO Yo la tengo y funciona bien.,Positivo
1493,Me encanta mi @DIRECTVGO 😍 donde voy tengo tod...,Positivo


# Preprocesamiento - Pipeline

In [118]:
from sklearn.pipeline import make_pipeline #Para realizar el preprocesamiento (limpieza de datos)
from sklearn.feature_extraction.text import TfidfVectorizer

import re
import string
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 

stopWords = set(stopwords.words('spanish'))
default_stopwords = stopwords.words('spanish') 
default_stemmer = PorterStemmer()

def clean_text(text, ):
    def tokenize_text(text):
        return [w for s in sent_tokenize(text) for w in word_tokenize(s)]

    def remove_stopwords(text, stop_words = default_stopwords):
        tokens = [w for w in tokenize_text(text) if w not in stop_words]
        return ' '.join(tokens)

    def remove_special_characters(text, characters = string.punctuation.replace('-', '')):
        tokens = tokenize_text(text)
        pattern = re.compile('[{}]'.format(re.escape(characters)))
        return ' '.join(filter(None, [pattern.sub('', t) for t in tokens]))

    def stem_text(text, stemmer = default_stemmer):
        tokens = tokenize_text(text)
        return ' '.join([stemmer.stem(t) for t in tokens])

    text = text.strip(' ')
    text = text.lower()
    text = stem_text(text) 
    text = remove_special_characters(text) 
    text = remove_stopwords(text)

    return text

**Realizo la limpieza al conjunto etiquetado df_train**

In [119]:
message_limpio = []
for i in df_train['Message']:
    message_limpio.append(clean_text(str(i)))

message_limpio[0:5]

['anda curao vidal xd',
 'ctm vidal tien 6 dedo',
 'carga vidal',
 'falsa resulta publicidad tien ver pelotero tatuado cort insufr pelo promoción est servicio publicista saben dónde parado aún viven década 60',
 'poquito hinchado falopa v']

**Distribución de la variable target**

In [120]:
serie_pos = df_train.Sentimiento.dropna().apply(lambda x: 1 if 'osit' in x else 0)
serie_neut = df_train.Sentimiento.dropna().apply(lambda x: 2 if 'eut' in x else 0)
serie_neg = df_train.Sentimiento.dropna().apply(lambda x: 3 if 'eg' in x else 0)
serie_final = serie_pos + serie_neut + serie_neg


df_train['sentimiento'] = serie_final
df_train['Message'] = [str (item) for item in message_limpio]
df_train = df_train[df_train.sentimiento != 0]
#df_train.dropna(inplace=True)

df_train.iloc[:,1:].groupby('sentimiento').count()

,Sentimiento
sentimiento,
1,538
2,6255
3,645


**Balanceo de la variable target**

In [121]:
postivos = df_train[df_train['sentimiento'] == 1]
neutros = df_train[df_train['sentimiento'] == 2]
negativos = df_train[df_train['sentimiento'] == 3]

df_train = pd.concat([postivos,negativos, neutros.sample(postivos.shape[0]*2)])
df_train.iloc[:,1:].groupby('sentimiento').count()

,Sentimiento
sentimiento,
1,538
2,1076
3,645


# Clasificación - Entrenamiento

In [122]:
# Separo los datos
X_train, X_valid, y_train, y_valid = train_test_split(df_train['Message'], 
                                                      df_train['sentimiento'], 
                                                      test_size = 0.20, 
                                                      stratify=df_train['sentimiento'], 
                                                      random_state = 12)

# Genero el modelo
model = make_pipeline(TfidfVectorizer(), MultinomialNB())
# Entreno el modelo
model.fit(X_train, y_train)
# Prediccion
y_pred = model.predict(X_valid) 
# Aciertos
print(accuracy_score(y_valid, y_pred))

0.8495575221238938


# Evaluación del modelo - Matriz de Confusión

In [123]:
# Matriz de confusion
confusion_matrix= cm(y_valid, y_pred)
list1 = ["Actual 1", "Actual 2", "Actual 3"]
list2 = ["Predicted 1", "Predicted 2", "Predicted 3"]
pd.DataFrame(confusion_matrix, list1,list2)

,Predicted 1,Predicted 2,Predicted 3
Actual 1,104,1,3
Actual 2,16,174,25
Actual 3,0,23,106


# Clasificación - Predicción

In [124]:
# Pruebo nuevamente con los datos no etiquetados df_test
message_limpio_test = []
for i in df_test['Message']:
    message_limpio_test.append(clean_text(str(i)))

df_test['Message'] = [str (item) for item in message_limpio_test]

# Realizo la predicción con el modelo ya entrenado
y_pred_sen = model.predict(df_test['Message']) 
df_test['Sentiment'] = y_pred_sen
df_test.loc[:,['Message', 'Sentiment']][df_test['Sentiment'] == 3][10:20]

,Message,Sentiment
30,aunqu digan dragón hizo má acertado comportami...,3
39,cállese gonorrea dracari pa est tontazo,3
40,hicieron perder año vida final mediocr falta c...,3
41,final tan malo regalado,3
65,peor final ctm😤😭 siempr político ase poder,3
68,toda seri cersi mala mató gent ahora resulta s...,3
80,fome final mató reina culpa enano sanza salió ...,3
82,fome fome peor final visto decepción,3
89,rey nocturno bien jajajaja final má miedo cerc...,3
100,aparec app store página direcciona descargarla...,3


**Resultados:**

In [125]:
df_test.groupby('Sentiment').count()['Message']

Sentiment
1     153
2    2550
3     570
Name: Message, dtype: int64

# Otra Opcion: Random Forest y GridSearchCV

In [126]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
from tempfile import mkdtemp
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from joblib import dump, load
from sklearn.metrics import classification_report

cache = mkdtemp()

pipe = Pipeline([('vec', CountVectorizer()), ('clf', RandomForestClassifier())], memory=cache)

params = [#{'clf': [MultinomialNB()],},
    {'clf': [RandomForestClassifier()], 
     'clf__n_estimators':[100, 200, 500],
     'clf__max_depth': [5, 10, 15],
     'clf__min_samples_split': [5, 10, 15],
     'vec': [CountVectorizer(), TfidfVectorizer()]
    }
]

gs = GridSearchCV(estimator=pipe, param_grid=params, cv=5, n_jobs=-1, verbose=10);

In [127]:
X_train, X_test, y_train, y_test = train_test_split(df_train['Message'], 
                                                    df_train['sentimiento'], 
                                                    test_size = 0.20, 
                                                    stratify=df_train['sentimiento'], 
                                                    random_state = 12)

gs.fit(X_train, y_train)
y_pred = gs.predict(X_test) 

dump(gs, 'modelo_dtv.joblib') ;

Fitting 5 folds for each of 54 candidates, totalling 270 fits
[CV 4/5; 1/54] START clf=RandomForestClassifier(), clf__max_depth=5, clf__min_samples_split=5, clf__n_estimators=100, vec=CountVectorizer()
[CV 4/5; 1/54] END clf=RandomForestClassifier(), clf__max_depth=5, clf__min_samples_split=5, clf__n_estimators=100, vec=CountVectorizer();, score=0.662 total time=   0.5s
[CV 3/5; 2/54] START clf=RandomForestClassifier(), clf__max_depth=5, clf__min_samples_split=5, clf__n_estimators=100, vec=TfidfVectorizer()
[CV 3/5; 2/54] END clf=RandomForestClassifier(), clf__max_depth=5, clf__min_samples_split=5, clf__n_estimators=100, vec=TfidfVectorizer();, score=0.645 total time=   0.6s
[CV 2/5; 3/54] START clf=RandomForestClassifier(), clf__max_depth=5, clf__min_samples_split=5, clf__n_estimators=200, vec=CountVectorizer()
[CV 2/5; 3/54] END clf=RandomForestClassifier(), clf__max_depth=5, clf__min_samples_split=5, clf__n_estimators=200, vec=CountVectorizer();, score=0.633 total time=   0.7s
[CV 1

['modelo_dtv.joblib']

In [128]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.93      0.92      0.92       108
           2       0.70      0.94      0.81       215
           3       0.93      0.40      0.56       129

    accuracy                           0.78       452
   macro avg       0.85      0.75      0.76       452
weighted avg       0.82      0.78      0.76       452



In [129]:
gs.best_estimator_

Pipeline(memory='/tmp/tmplffjzry_',
         steps=[('vec', CountVectorizer()),
                ('clf',
                 RandomForestClassifier(max_depth=15, min_samples_split=10,
                                        n_estimators=200))])

## Comparación de Resultados:

In [130]:
gs2 = load('modelo_dtv.joblib') 
y_pred_sen = gs2.predict(df_test['Message'])
df_test['Sentiment_RF'] = y_pred_sen
print("Bayes: \n\n", df_test.groupby("Sentiment").size())
print("\n\nRandom Forest: \n\n", df_test.groupby("Sentiment_RF").size())

Bayes: 

 Sentiment
1     153
2    2550
3     570
dtype: int64


Random Forest: 

 Sentiment_RF
1      51
2    3163
3      59
dtype: int64


# Otra Opción de Preprocesamiento: Spacy

In [131]:
#!spacy download es_core_news_sm

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 11.3 MB/s eta 0:00:0000:010:01
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [132]:
import spacy

nlp = spacy.load('es_core_news_sm')

df.loc[:,'corpus_spacy_doc'] = df['Message'].apply(lambda cell: nlp(str(cell)))

def remove_stops(tokens):
    return [token.text for token in tokens if not token.is_stop]

df['No Stop'] = df['corpus_spacy_doc'].apply(remove_stops)
df["final"] = df['No Stop'].apply(lambda row: " ".join([w.lemma_ for w in nlp(str(row))]))  #Lemmas
df["final"] = df.final.apply(lambda text: " ".join(token.lemma_ 
                                                   for token in nlp(text)
                                                   if not token.is_punct))   #Elimino no alfanuméricos (\W)
df["final"][15:20]


[CV 3/5; 45/54] END clf=RandomForestClassifier(), clf__max_depth=15, clf__min_samples_split=10, clf__n_estimators=200, vec=CountVectorizer();, score=0.759 total time=   0.8s
[CV 2/5; 46/54] START clf=RandomForestClassifier(), clf__max_depth=15, clf__min_samples_split=10, clf__n_estimators=200, vec=TfidfVectorizer()
[CV 2/5; 46/54] END clf=RandomForestClassifier(), clf__max_depth=15, clf__min_samples_split=10, clf__n_estimators=200, vec=TfidfVectorizer();, score=0.751 total time=   0.8s
[CV 1/5; 47/54] START clf=RandomForestClassifier(), clf__max_depth=15, clf__min_samples_split=10, clf__n_estimators=500, vec=CountVectorizer()
[CV 1/5; 47/54] END clf=RandomForestClassifier(), clf__max_depth=15, clf__min_samples_split=10, clf__n_estimators=500, vec=CountVectorizer();, score=0.749 total time=   1.9s
[CV 5/5; 47/54] START clf=RandomForestClassifier(), clf__max_depth=15, clf__min_samples_split=10, clf__n_estimators=500, vec=CountVectorizer()
[CV 5/5; 47/54] END clf=RandomForestClassifier()

15                                  importar Arturo haí
16                    Borracho mario krauzberguer plato
17    Borracho pegar 1000 patas rajar ctm envidioso ...
18                                perro vidal chile rey
19    @directvgoayuda Hola cliente plan oro 4 k abon...
Name: final, dtype: object